In [1]:
from statsmodels.stats.proportion import proportion_confint

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [10]:
n1 = 11037
n2 = 11034
p1 = 104.0/n1
p2 = 189.0/n2
print p2 - p1

0.007706023976


Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки. 

## Доверительный интервал для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [12]:
import scipy
import numpy as np

In [13]:
alpha = 0.05    
z = scipy.stats.norm.ppf(1 - alpha / 2.)
left_boundary = (p2 - p1) - z * np.sqrt(p1 * (1 - p1)/n1 + p2 * (1 - p2)/n2)
right_boundary = (p2 - p1) + z * np.sqrt(p1 * (1 - p1)/n1 + p2 * (1 - p2)/n2)

In [16]:
print "Доверительный интервал: [%f, %f]" % (left_boundary, right_boundary)

Доверительный интервал: [0.004688, 0.010724]


Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину p/(1−p), которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить p^. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

189/11034/(1−189/11034)=189/(11034−189)≈0.0174

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки. 

In [17]:
odds1 = p1/(1 - p1)
odds2 = p2/(1 - p2)
print odds2/odds1

1.83205394191


Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

   * составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
   * установите random seed=0;
   * сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [31]:
vector1 = np.zeros(11037)
vector1[:104] = 1

vector2 = np.zeros(11034)
vector2[:189] = 1

In [33]:
print sum(vector1), sum(vector2)

104.0 189.0


In [51]:
def calculate_odds1(vector):
    n = sum(vector)
    return n/(11037 - n)

In [52]:
def calculate_odds2(vector):
    n = sum(vector)
    return n/(11034 - n)

In [40]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [41]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [63]:
np.random.seed(0)

odds1_scores = map(calculate_odds1, get_bootstrap_samples(vector1, 1000))
odds2_scores = map(calculate_odds2, get_bootstrap_samples(vector2, 1000))

odds_relation = []

for i in range(1000):
    odds_relation.append(odds2_scores[i]/odds1_scores[i])

print "95% confidence interval for odds_relation:",  stat_intervals(odds_relation, 0.05)

95% confidence interval for odds_relation: [ 1.44419465  2.34321168]
